# SVM

In [1]:
%load_ext autoreload
%autoreload 2
from preprocessing import *
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
from joblib import dump, load

In [2]:
SCORINGS = ["f1", "roc_auc", "accuracy", "recall", "precision"]
METRIC = "roc_auc"

def tabla(grid, params):
    tabla = pd.DataFrame(grid.cv_results_)
    tabla.sort_values("rank_test_" + METRIC, inplace = True)
    tabla.reset_index(inplace = True)
    cols = ["param_svc__" + x for x in params] + ["mean_test_" + x for x in SCORINGS]
    return tabla[cols]

## SVM Lineal/Polinómico (Grado 1, 2 y 3)

In [3]:
# Este modelo tarda mucho en entrenar (muchos parámetros y K-fold, y SVM es lento), si se desea se puede
# cargar de archivo descomentando esta línea
# grid = load('SVM/polinomico.joblib') 

In [4]:
initialize_dataset()
df_features = pd.read_csv("datasets/df_features.csv", low_memory = False, index_col = "id")
df_target = pd.read_csv("datasets/df_target.csv", low_memory=False, index_col = "id")

initialize_dataset()
common(df_features, df_target)
viento_trigonometrico(df_features)
# El barrio tiene 49 valores distintos. Para no tener que hacer one hoy con 48 columnas nuevas, uso hashing trick
df_features = hashing_trick(df_features, 24, "barrio")
pipe = standarizer()
pipe = simple_imputer(pipe)

In [5]:
pipe.steps.append(('svc', SVC(kernel = 'poly', random_state = 123, max_iter=100000)))

# Usamos Grid search y cross validation. Decidimos en este caso usar el valor de gamma por defecto, ya que el entrenamiento tarda muchísimo
grid = GridSearchCV(pipe, param_grid = {"svc__C": [0.01, 1, 1000], "svc__coef0": [1, 1000], "svc__degree": [1, 2, 3]},
                    verbose = 1, n_jobs = -1, cv = StratifiedKFold(3), scoring = SCORINGS, refit = METRIC)

grid.fit(df_features, df_target.values.ravel())

grid.best_score_

Fitting 3 folds for each of 18 candidates, totalling 54 fits


/home/delu/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/delu/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/delu/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/delu/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/delu/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver termi

0.8815153624472941

In [6]:
tabla(grid, ["C", "degree", "coef0"])

,param_svc__C,param_svc__degree,param_svc__coef0,mean_test_f1,mean_test_roc_auc,mean_test_accuracy,mean_test_recall,mean_test_precision
0,1,3,1,0.624965,0.881515,0.859974,0.521124,0.780518
1,0.01,3,1,0.573322,0.879088,0.851238,0.446404,0.801092
2,1,2,1,0.606994,0.876067,0.856358,0.495461,0.783370
3,0.01,2,1000,0.586134,0.874960,0.850837,0.471761,0.773756
4,0.01,2,1,0.551435,0.874840,0.846303,0.421962,0.795542
5,1,1,1000,0.568219,0.863662,0.843479,0.460021,0.742978
6,1,1,1,0.568280,0.863661,0.843519,0.460021,0.743190
7,0.01,1,1000,0.509199,0.861549,0.837060,0.377531,0.781898
8,0.01,1,1,0.509213,0.861549,0.837050,0.377575,0.781776
9,1000,1,1000,0.384516,0.619271,0.620943,0.528196,0.302372


Podemos ver que hay varios modelos en los que no terminó de converger tras 100000 iteraciones, y saltaron todas las alertas (sugieren que estandaricemos los datos. Estos son en su mayoría los que tienen C=1000 (se puede notar en la tabla viendo que su score promedio en el fold es pésimo). También dieron bastante mal algunas otras combinaciones de parámetros degrado 2 y 3.

El mejor en este caso resultó ser el del kernel de grado 3, C=1 y coef0=1. Dio un accuracy de alrededor de 86,0% en promedio y área bajo la curva ROC de 0,882. Se puede notar commo (de los que lograron resultados razonables), estan ordenados de mayor grado a menor. Esto es razonable ya que a más grado menos sesgado esta, y confiamos que no overfittearon al haber hecho K-Fold.

El recall es bastante malo, lo que es de esperar siendo que el dataset tiene mucho más negativos que positivos. Tiende a predecir por la negativa, dando como resultado muchos falsos negativos y en consecuencia el bajo recall.

In [7]:
dump(grid, 'SVM/polinomico.joblib') 

['SVM/polinomico.joblib']

## SVM Radial (RBF)

In [8]:
# Este modelo tarda mucho en entrenar (muchos parámetros y K-fold, y SVM es lento), si se desea se puede
# cargar de archivo descomentando esta línea
# grid2 = load('SVM/radial.joblib') 

In [9]:
initialize_dataset()
df_features = pd.read_csv("datasets/df_features.csv", low_memory = False, index_col = "id")
df_target = pd.read_csv("datasets/df_target.csv", low_memory=False, index_col = "id")

initialize_dataset()
common(df_features, df_target)
viento_trigonometrico(df_features)
# El barrio tiene 49 valores distintos. Para no tener que hacer one hoy con 48 columnas nuevas, uso hashing trick
df_features = hashing_trick(df_features, 24, "barrio")
pipe = standarizer()
pipe = simple_imputer(pipe)

In [10]:
pipe.steps.append(('svc', SVC(kernel = 'rbf', random_state = 123, max_iter=100000)))

grid2 = GridSearchCV(pipe, param_grid = {"svc__C": [0.01, 1, 1000], "svc__gamma": [0.00001, 0.001, 0.1]}, \
                     verbose = 1, n_jobs = -1, cv = StratifiedKFold(3), scoring = SCORINGS, refit = METRIC)

grid2.fit(df_features, df_target.values.ravel())

grid2.best_score_

Fitting 3 folds for each of 9 candidates, totalling 27 fits


/home/delu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/delu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/delu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/delu/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated

0.8709781744515421

In [11]:
tabla(grid2, ["C", "gamma"])

,param_svc__C,param_svc__gamma,mean_test_f1,mean_test_roc_auc,mean_test_accuracy,mean_test_recall,mean_test_precision
0,1,0.001,0.571085,0.870978,0.846909,0.455220,0.766071
1,1,0.1,0.617908,0.866318,0.856192,0.519378,0.762727
2,1000,0.00001,0.570394,0.865050,0.844330,0.461592,0.746313
3,0.01,0.1,0.000000,0.862972,0.776114,0.000000,0.000000
4,1000,0.001,0.617482,0.859594,0.855586,0.521777,0.759685
5,0.01,0.001,0.013330,0.858084,0.777326,0.006721,0.844697
6,1,0.00001,0.018900,0.857499,0.777873,0.009558,0.850942
7,0.01,0.00001,0.000000,0.856695,0.776114,0.000000,0.000000
8,1000,0.1,0.578097,0.825440,0.818503,0.555386,0.602798


En este caso caso la mayoría de los parámetros no logró converger antes de las iteraciones dadas. Casi todos los modelos dieron 77% de accuracy, lo cual es muy malo ya que probablemente hayan fiteado a decir siempre que "no" al estar desbalanceada la variable target. El hecho de que haya algunos con recall y precision en 0 da a entender lo mismo (0 true positives, entonces 0 de recall y precision).

Los primeros 3 si dieron valores más aceptables, siendo el de mejor el que usa C=1 y Gamma=0,001 (0.872 de área bajo curva ROC y accuracy del 84,8%).

Aún así, dio peor que el kernel polinómico, y el recall es bastante malo para el primer modelo, lo cual quiere decir que tiene muchos falsos negativos. El segundo modelo es levemente mejor en ese aspecto, aunque tiene peor área bajo la curva ROC. Como el dataset esta desbalanceado, tiende más a la negativa por lo que es esperable que tenga mejor precisión que recall.

In [12]:
dump(grid2, 'SVM/radial.joblib') 

['SVM/radial.joblib']